In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import ProgbarLogger, EarlyStopping

##Data Preparation


In [52]:
# Functions I found to help with data prep
# https://medium.com/@john.kosinski/preparing-and-shaping-timeseries-data-for-keras-lstm-input-part-two-ad17f6ab450

# extract X with the given number of timesteps
# df: the DataFrame
# ntimesteps: number of timesteps
#
def extract_X(df: pd.DataFrame, ntimesteps: int):
    features = len(df.columns)
    X = list()

    #offset for timesteps
    offsets = list()
    for i in range (ntimesteps, 0, -1):
        offsets.append(df.shift(i))

    #combine timestep columns into rows
    combined = pd.concat(offsets, axis=1)
    combined = combined.tail(-ntimesteps)
    combined.drop(combined.tail(1).index, inplace=True)

    #reshape each row (timesteps, features)
    for i in range(len(combined)):
        row = combined.iloc[i].to_numpy()
        xrow = list()
        for n in range(ntimesteps):
            xrow.append(row[n*features:(n*features)+features])
        X.append(xrow)

    #return as numpy array
    return np.array(X)

# extract y column (the col to be predicted)
# df: the DataFrame
# col_name: the name of the column to be predicted
# ntimesteps: number of timesteps
#
def extract_y(df: pd.DataFrame, col_name: str, ntimesteps: int):
    shifted = df.shift(-1)
    shifted = shifted.head(-2)
    shifted = shifted.tail(-(ntimesteps-1))
    return shifted[col_name].values

class DataSet:
    def __init__(self, X, y):
        if X.ndim != 3:
            raise Exception("Expected a 3-dimensional array for X")
        if y.ndim != 1:
            raise Exception("Expected a 1-dimensional array for y")
        if len(X) != len(y):
            raise Exception("Length of X and y must be the same")

        self.X = X
        self.y = y

    def train_test_split(self, test_size=0.2):
        # Calculate the number of samples for testing
        num_samples = len(self.X)
        num_test_samples = int(num_samples * test_size)

        # Shuffle indices
        indices = np.random.permutation(num_samples)

        # Split indices into training and testing
        test_indices = indices[:num_test_samples]
        train_indices = indices[num_test_samples:]

        # Split X and y based on indices
        X_train, y_train = self.X[train_indices], self.y[train_indices]
        X_test, y_test = self.X[test_indices], self.y[test_indices]

        # Create DataSet objects for training and testing sets
        train_data_set = DataSet(X_train, y_train)
        test_data_set = DataSet(X_test, y_test)

        return train_data_set, test_data_set

    @property
    def size(self):
        return len(self.X)

In [53]:
# load the dataset
df = pd.read_csv('train.csv')

In [54]:
# drop na values
df.dropna()

# calulate ratio
df['bid_ask_ratio'] = df['bid_size']/df['ask_size']

# create target variable
df["target_change"] = np.ones_like(df["target"])
df.loc[df["target"]<0,'target_change'] = -1

# Select features and target
selected_features = ['stock_id', 'ask_size', 'imbalance_size', 'matched_size', 'bid_size', 'bid_ask_ratio', 'reference_price']

# use only stock_id 0
df = df[df['stock_id'] == 0]
df_selected = df[selected_features]
target = df[['date_id', 'target_change']]

df_selected
target

,date_id,target_change
0,0,-1.0
191,0,1.0
382,0,1.0
573,0,1.0
764,0,1.0
...,...,...
5236980,480,1.0
5237180,480,1.0
5237380,480,-1.0
5237580,480,1.0


In [55]:
# # Normalize the data
# scaler = MinMaxScaler()
# scaled_data = scaler.fit_transform(df_selected)
# # Convert scaled data back to a DataFrame
# scaled_df = pd.DataFrame(data=scaled_data, columns=df_selected.columns)

# Format data for LSTM input (sample/batches, timesteps, features)
dataset = DataSet(extract_X(df_selected, 5), extract_y(target, 'target_change', 5))
dataset.X.shape

(26449, 5, 7)

In [56]:
# Split into train and test
tr_data, te_data = dataset.train_test_split(test_size = .2)
tr_data.X.shape

(21160, 5, 7)

## LSTM Model


In [57]:
# Define model
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(5, 7)))
model.add(Dense(1, activation='tanh')) # Output layer

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Fit model
model.fit(tr_data.X, tr_data.y, epochs=100, verbose=1)

Epoch 1/100
662/662 [==============================] - 11s 10ms/step - loss: 2.0221 - accuracy: 0.4594
Epoch 2/100
662/662 [==============================] - 5s 7ms/step - loss: 2.0221 - accuracy: 0.4594
Epoch 3/100
662/662 [==============================] - 6s 8ms/step - loss: 2.0221 - accuracy: 0.4594
Epoch 4/100
662/662 [==============================] - 8s 13ms/step - loss: 2.0221 - accuracy: 0.4594
Epoch 5/100
662/662 [==============================] - 7s 10ms/step - loss: 2.0221 - accuracy: 0.4594
Epoch 6/100
662/662 [==============================] - 6s 9ms/step - loss: 2.0221 - accuracy: 0.4594
Epoch 7/100
662/662 [==============================] - 7s 11ms/step - loss: 2.0221 - accuracy: 0.4594
Epoch 8/100
662/662 [==============================] - 6s 9ms/step - loss: 2.0221 - accuracy: 0.4594
Epoch 9/100
662/662 [==============================] - 7s 11ms/step - loss: 2.0221 - accuracy: 0.4594
Epoch 10/100
662/662 [==============================] - 5s 8ms/step - loss: 2.0221 - 

##Test Model

In [58]:
loss, accuracy = model.evaluate(te_data.X, te_data.y)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

166/166 [==============================] - 2s 6ms/step - loss: 2.0503 - accuracy: 0.4511
Test Loss: 2.05029296875
Test Accuracy: 0.45112496614456177
